# modeling10-nli-binaryclassifier-FiDEncoder-loading
- Checking parser
- modeling based on FiDEncoder
- checking metric

## CHECKING PARSER

In [1]:
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import set_seed
import heapq
import pathlib
import shutil
from FiD.src.model import FiDT5
from src.model import FiDEncoderForSequenceClassification

from pprint import pprint
from tqdm.auto import tqdm
from src.data import BinaryCustomDatasetShuffle

import json
import math
import os
import logging
import sys
import evaluate
from util import utils

import transformers
import torch
import numpy as np
import random
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoModelForSequenceClassification,
    AutoModel,
    AutoConfig,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    set_seed,
    get_scheduler,
)
from util.arguments import ModelArguments, DataTrainingArguments, CustomTrainingArguments

In [2]:
parser = HfArgumentParser((ModelArguments, DataTrainingArguments, CustomTrainingArguments))

model_args, data_args, train_args = parser.parse_args_into_dataclasses([])

In [3]:
vars(model_args)

{'model_architecture': 'roberta-large',
 'model_name_or_path': 'roberta-large',
 'config_base_path': '/data/philhoon-relevance/FiD/pretrained_models/nq_reader_large',
 'prediction_model_name_or_path': '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/roberta-decisive_binary_gold_data_trial1',
 'prediction_model_step': '380',
 'git_tag': 'v1.1',
 'config_name': None,
 'tokenizer_name': None,
 'max_seq_length': 200}

In [4]:
vars(data_args)

{'data': 'NQ-DEV-DPR/5-fold/1',
 'train_file': '/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/binary_data/binary_ex_ctx100id_split_train_1_partial.json',
 'eval_file': '/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/binary_data/binary_ex_ctx100id_split_train_1_partial.json',
 'intact_eval': False,
 'num_labels': 2,
 'overwrite_cache': False,
 'pad_to_max_length': False,
 'dataset_class': 'BinaryCustomDatasetShuffle'}

In [5]:
train_dict = vars(train_args)

In [6]:
train_dict

{'do_train': True,
 'do_eval': True,
 'do_predict': False,
 'with_tracking': True,
 'report_to': 'wandb',
 'wandb_project': 'binary_classifier',
 'run_name': 'testing_from_scratch',
 'output_dir': '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/testing-scratch',
 'per_device_train_batch_size': 8,
 'per_device_eval_batch_size': 8,
 'seed': 42,
 'learning_rate': 5e-05,
 'weight_decay': 0.0,
 'adam_beta1': 0.9,
 'adam_beta2': 0.999,
 'adam_epsilon': 1e-08,
 'lr_scheduler_type': 'linear',
 'gradient_accumulation_steps': 1,
 'max_train_steps': None,
 'num_train_epochs': 3,
 'num_warmup_steps': 0,
 'checkpointing_steps': None,
 'train_loss_steps': 10,
 'save_max_limit': 3,
 'best_metric': 'accuracy',
 'headtype': 'adaptive',
 'class_weights': False,
 'num_layers': None,
 'drop_out_rate': None,
 'padding': -100}

## modeling

In [7]:
from pprint import pprint
import numpy as np
import torch
from torch import nn
import transformers
from transformers import AutoConfig, AutoTokenizer
from transformers import T5PreTrainedModel
import copy
from transformers.modeling_outputs import SequenceClassifierOutput
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

In [8]:
from FiD.src.model import FiDT5

In [7]:
model_path = '/data/philhoon-relevance/FiD/pretrained_models/nq_reader_large'
num_labels = 2

In [8]:
model_class = FiDT5

In [9]:
config = AutoConfig.from_pretrained(model_path, num_labels=num_labels)

In [10]:
pprint(config)

T5Config {
  "_name_or_path": "/data/philhoon-relevance/FiD/pretrained_models/nq_reader_large",
  "architectures": [
    "FiDT5"
  ],
  "d_ff": 4096,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 30

In [13]:
model = model_class.from_pretrained(model_path)

In [14]:
# model.encoder => FiDT5.EncoderWrapper
# model.encoder.encoder => FiDT5.EncoderWrapper.encoder = T5 encoder Architecture w FiDT5 parameters
model_encoder = model.encoder.encoder

In [15]:
type(model_encoder)

transformers.models.t5.modeling_t5.T5Stack

In [11]:
tokenizer2 = AutoTokenizer.from_pretrained('t5-base', return_dict=False)

/home/philhoon/miniconda3/envs/relevance-kilt/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
# tokenize

In [17]:
tokenizer = transformers.T5Tokenizer.from_pretrained('t5-base', return_dict=False)

/home/philhoon/miniconda3/envs/relevance-kilt/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:164: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [12]:
example = {'id': 12, 
           'question': "when was the public service commission original version of the upsc set up", 
           'ctx': {"id": "17105334", 
                   "title": "Bihar Public Service Commission", 
                   "text": "3 of the Regulations, 1960 the Commission was constituted with a Chairman and 10 (ten) other members. The strength of members was reduced to 6 (six) after bifurcation of the State of Bihar and the State of Jharkhand vide notification no. 7/PSC-1013/95 (Part-3) Per 8262 dated 9 October 2002 of the Personnel & Administrative Reforms Department, Bihar. Article 320 and 321 of the Constitution of India prescribes the mandate of the State Public Service Commissions, which are: a)Recruitment by conduct of Competitive Examinations/ through interviews to the services of the State Government. b)Advising the State Government on the suitability of"}
          }


In [18]:
padded_output = tokenizer2(example['question'], padding=True)

In [22]:
padded_output.convert_ids_to_tokens(padded_output['input_ids'])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/philhoon/miniconda3/envs/relevance-kilt/lib/python3.8/site-packages/transformers/tokenizat │
│ ion_utils_base.py:248 in __getattr__                                                             │
│                                                                                                  │
│    245 │                                                                                         │
│    246 │   def __getattr__(self, item: str):                                                     │
│    247 │   │   try:                                                                              │
│ ❱  248 │   │   │   return self.data[item]                                                        │
│    249 │   │   except KeyError:                                                                  │
│    250 │   │   │   raise AttributeError                                                          │
│    251                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'convert_ids_to_tokens'

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_2662967/3234877771.py:1 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2662967/3234877771.py'                      │
│                                                                                                  │
│ /home/philhoon/miniconda3/envs/relevance-kilt/lib/python3.8/site-packages/transformers/tokenizat │
│ ion_utils_base.py:250 in __getattr__                                                             │
│                                                                                                  │
│    247 │   │   try:                                                                              │
│    248 │   │   │   return self.data[item]                                                        │
│    249 │   │   except KeyError:                                                                  │
│ ❱  250 │   │   │   raise AttributeError                                                          │
│    251 │                                                                                         │
│    252 │   def __getstate__(self):                                                               │
│    253 │   │   return {"data": self.data, "encodings": self._encodings}                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError

In [16]:
tokenizer2.pad_token

'<pad>'

In [ ]:
tokenizer2.convert_to_id

In [20]:
test_ids = np.expand_dims(np.array(output['input_ids']), 0)

In [21]:
test_ids = torch.from_numpy(test_ids)
print(test_ids)

tensor([[ 116,   47,    8,  452,  313, 5473,  926,  988,   13,    8,   95,    7,
           75,  356,   95,    1]])


In [22]:
test_attention = np.expand_dims(np.array(output['attention_mask']), 0)

In [23]:
test_attention = torch.from_numpy(test_attention)
print(test_attention)

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [24]:
result = model_encoder.forward(input_ids = test_ids, attention_mask = test_attention)

In [25]:
vars(result)

{'last_hidden_state': tensor([[[-0.2608,  0.2637, -0.5229,  ...,  0.0836,  0.0304, -0.2207],
          [-0.0648,  0.0149, -0.0261,  ..., -0.0078,  0.0553, -0.0412],
          [-0.2228, -0.2095,  0.0784,  ..., -0.1607,  0.1964,  0.2240],
          ...,
          [-0.2896, -0.4332, -0.0921,  ..., -0.2269,  0.0741,  0.0176],
          [-0.1534, -0.3360,  0.3193,  ..., -0.1065,  0.0356,  0.5194],
          [ 0.1150,  0.1222, -0.1723,  ...,  0.3307, -0.2283, -0.2829]]],
        grad_fn=<MulBackward0>),
 'past_key_values': None,
 'hidden_states': None,
 'attentions': None,
 'cross_attentions': None}

In [26]:
result['last_hidden_state'].size()

torch.Size([1, 16, 1024])

In [27]:
config.num_labels

2

In [28]:
class ClassificationHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.d_model, config.d_model)
        classifier_dropout = (
            config.dropout_rate
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.out_proj = nn.Linear(config.d_model, config.num_labels)

    def forward(self, features, **kwargs):
        x = features[:, 0, :]  # take <s> token (equiv. to [CLS])
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

In [29]:
class AdaptivePooler(nn.Module):
    """ Calcualte weighted average of the inputs with learnable weights """
    def __init__(self, config):
        super().__init__()
        self.input_size = config.d_model
        self.w = nn.Linear(self.input_size, 1, bias=True)
        self.dropout = nn.Dropout(config.dropout_rate)
        self.classifier = nn.Linear(config.d_model, config.num_labels)

    def forward(self, inputs, mask=None):
        batch_size, seq_len, emb_dim = inputs.shape
        scores = torch.squeeze(self.w(inputs), dim=-1)
        weights = nn.functional.softmax(scores, dim=-1)
        if mask is not None:
            weights = weights * mask
            weights = weights / weights.sum(dim=-1, keepdims=True)
        outputs = (inputs.permute(2, 0, 1) * weights).sum(-1).T
        
        outputs = self.dropout(outputs)
        logits = self.classifier(outputs)
        
        return logits

In [30]:
def init_weights_test(module):
    """ Initialize the weights """
    if isinstance(module, (nn.Linear, nn.Embedding)):
        module.weight.data.zero_()
       
    if isinstance(module, nn.Linear) and module.bias is not None:
        module.bias.data.zero_()

In [31]:
# adaptivePooler_.apply(init_weights_test)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_378934/447010445.py:1 in <module>                                                 │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_378934/447010445.py'                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'adaptivePooler_' is not defined

In [33]:
# adaptivePooler_.w.weight

In [34]:
config.problem_type

In [ ]:
class FiDEncoderForSequenceClassification(nn.Module):
    def __init__(self, config, model_encoder, pooler='adaptive'):
        super(FiDEncoderForSequenceClassification, self).__init__()
        self.num_labels = config.num_labels
        self.config = config
        
        self.encoder = model_encoder
        
        classification_class = AdaptivePooler if pooler == 'adaptive' else ClassificationHead
        self.classifier = classification_class(self.config)
        
        self.classifier.apply(self._init_weights)
        
    def _init_weights(self, module):
        """ Initialize the weights """
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=config.initializer_factor)
        if isinstance(module, nn.Linear) and module.bias is not None:
            module.bias.data.zero_()
            
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        head_mask=None,
        inputs_embeds=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        labels=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`):
            Labels for computing the sequence classification/regression loss. Indices should be in :obj:`[0, ...,
            config.num_labels - 1]`. If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """

        outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        
        logits = self.classifier(outputs[0], mask=attention_mask)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)
        
        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [ ]:
encoder_classifier = FiDEncoderForSequenceClassification(config, model_encoder)

In [ ]:
outputs = encoder_classifier.forward(input_ids = test_ids, attention_mask = test_attention, return_dict = True)

In [ ]:
outputs

In [ ]:
no_decay = ["bias", "layer_norm.weight"]

In [ ]:
param = [p for n, p in encoder_classifier.named_parameters() if not any(nd in n for nd in no_decay)]

In [ ]:
param

In [ ]:
for n, p in model.named_parameters() :
    if any(nd in n for nd in no_decay):
        print(n)

# Loading Model

In [37]:
encoder_classifier = FiDEncoderForSequenceClassification(config, model_encoder)

In [50]:
# encoder_classifier.state_dict()

In [38]:
model_path = '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/fid-encoder-adpative-decisive-trial1/step_260/pytorch_model.bin'

In [51]:
state_dict_ = torch.load(model_path)

In [40]:
type(model)

collections.OrderedDict

In [52]:
encoder_classifier.load_state_dict(state_dict_)

<All keys matched successfully>

In [54]:
# encoder_classifier.state_dict()

In [55]:
# encoder_classifier.eval()

FiDEncoderForSequenceClassification(
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): CheckpointWrapper(
        (module): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=1024, out_features=1024, bias=False)
                (k): Linear(in_features=1024, out_features=1024, bias=False)
                (v): Linear(in_features=1024, out_features=1024, bias=False)
                (o): Linear(in_features=1024, out_features=1024, bias=False)
                (relative_attention_bias): Embedding(32, 16)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=1024, out_features=4096, bias=False)
                (wo): Linear(in_features=4096, out_

In [ ]:
1.0746,  0.5811

# Checking Metric

In [24]:
metric_acc = evaluate.load("accuracy")
metric_pre = evaluate.load('precision')
metric_re = evaluate.load('recall')
metric_f1 = evaluate.load('f1')

In [25]:
predictions = np.array([1,1,1,1,1])
references = np.array([1,1,1,0,0])

In [26]:
metric_acc.add_batch(
                predictions=predictions,
                references=references,
            )

In [27]:
eval_metric = metric_acc.compute()
print(eval_metric)

{'accuracy': 0.6}


In [28]:
eval_metric = metric_acc.compute()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_56906/327186789.py:1 in <module>                                                  │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_56906/327186789.py'                         │
│                                                                                                  │
│ /home/philhoon/miniconda3/envs/relevance-kilt/lib/python3.8/site-packages/evaluate/module.py:433 │
│ in compute                                                                                       │
│                                                                                                  │
│   430 │   │                                                                                      │
│   431 │   │   if any(v is not None for v in inputs.values()):                                    │
│   432 │   │   │   self.add_batch(**inputs)                                                       │
│ ❱ 433 │   │   self._finalize()                                                                   │
│   434 │   │                                                                                      │
│   435 │   │   self.cache_file_name = None                                                        │
│   436 │   │   self.filelock = None                                                               │
│                                                                                                  │
│ /home/philhoon/miniconda3/envs/relevance-kilt/lib/python3.8/site-packages/evaluate/module.py:385 │
│ in _finalize                                                                                     │
│                                                                                                  │
│   382 │   │                                                                                      │
│   383 │   │   elif self.process_id == 0:                                                         │
│   384 │   │   │   # Let's acquire a lock on each node files to be sure they are finished writi   │
│ ❱ 385 │   │   │   file_paths, filelocks = self._get_all_cache_files()                            │
│   386 │   │   │                                                                                  │
│   387 │   │   │   # Read the predictions and references                                          │
│   388 │   │   │   try:                                                                           │
│                                                                                                  │
│ /home/philhoon/miniconda3/envs/relevance-kilt/lib/python3.8/site-packages/evaluate/module.py:302 │
│ in _get_all_cache_files                                                                          │
│                                                                                                  │
│   299 │   │   """                                                                                │
│   300 │   │   if self.num_process == 1:                                                          │
│   301 │   │   │   if self.cache_file_name is None:                                               │
│ ❱ 302 │   │   │   │   raise ValueError(                                                          │
│   303 │   │   │   │   │   "Evaluation module cache file doesn't exist. Please make sure that y   │
│   304 │   │   │   │   │   "at least once before calling `compute`."                              │
│   305 │   │   │   │   )                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Evaluation module cache file doesn't exist. Please make sure that you call `add` or `add_batch` at 
least once before calling `compute`.

In [17]:
eval_metric

{'accuracy': 0.6}

In [ ]:
import types
import torch
import transformers
import torch.nn.functional as F
from torch import nn
from torch.nn import CrossEntropyLoss
import numpy as np

class FiDT5(transformers.T5ForConditionalGeneration):
    def __init__(self, config):
        super().__init__(config)
        self.wrap_encoder()

    def forward_(self, **kwargs):
        if 'input_ids' in kwargs:
            kwargs['input_ids'] = kwargs['input_ids'].view(kwargs['input_ids'].size(0), -1)
        if 'attention_mask' in kwargs:
            kwargs['attention_mask'] = kwargs['attention_mask'].view(kwargs['attention_mask'].size(0), -1)

        return super(FiDT5, self).forward(
            **kwargs
        )

    # We need to resize as B x (N * L) instead of (B * N) x L here
    # because the T5 forward method uses the input tensors to infer
    # dimensions used in the decoder.
    # EncoderWrapper resizes the inputs as (B * N) x L.
    def forward(self, input_ids=None, attention_mask=None, **kwargs):
        if input_ids != None:
            # inputs might have already be resized in the generate method
            if input_ids.dim() == 3:
                self.encoder.n_passages = input_ids.size(1)
            input_ids = input_ids.view(input_ids.size(0), -1)
        if attention_mask != None:
            attention_mask = attention_mask.view(attention_mask.size(0), -1)
        return super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )

    # We need to resize the inputs here, as the generate method expect 2D tensors
    def generate(self, input_ids, attention_mask, max_length):
        self.encoder.n_passages = input_ids.size(1)
        return super().generate(
            input_ids=input_ids.view(input_ids.size(0), -1),
            attention_mask=attention_mask.view(attention_mask.size(0), -1),
            max_length=max_length
        )

    def wrap_encoder(self, use_checkpoint=False):
        """
        Wrap T5 encoder to obtain a Fusion-in-Decoder model.
        """
        self.encoder = EncoderWrapper(self.encoder, use_checkpoint=use_checkpoint)

    def unwrap_encoder(self):
        """
        Unwrap Fusion-in-Decoder encoder, useful to load T5 weights.
        """
        self.encoder = self.encoder.encoder
        block = []
        for mod in self.encoder.block:
            block.append(mod.module)
        block = nn.ModuleList(block)
        self.encoder.block = block

    def load_t5(self, state_dict):
        self.unwrap_encoder()
        self.load_state_dict(state_dict)
        self.wrap_encoder()

    def set_checkpoint(self, use_checkpoint):
        """
        Enable or disable checkpointing in the encoder.
        See https://pytorch.org/docs/stable/checkpoint.html
        """
        for mod in self.encoder.encoder.block:
            mod.use_checkpoint = use_checkpoint

    def reset_score_storage(self):
        """
        Reset score storage, only used when cross-attention scores are saved
        to train a retriever.
        """
        for mod in self.decoder.block:
            mod.layer[1].EncDecAttention.score_storage = None

    def get_crossattention_scores(self, context_mask):
        """
        Cross-attention scores are aggregated to obtain a single scalar per
        passage. This scalar can be seen as a similarity score between the
        question and the input passage. It is obtained by averaging the
        cross-attention scores obtained on the first decoded token over heads,
        layers, and tokens of the input passage.

        More details in Distilling Knowledge from Reader to Retriever:
        https://arxiv.org/abs/2012.04584.
        """
        scores = []
        n_passages = context_mask.size(1)
        for mod in self.decoder.block:
            scores.append(mod.layer[1].EncDecAttention.score_storage)
        scores = torch.cat(scores, dim=2)
        bsz, n_heads, n_layers, _ = scores.size()
        # batch_size, n_head, n_layers, n_passages, text_maxlength
        scores = scores.view(bsz, n_heads, n_layers, n_passages, -1)
        scores = scores.masked_fill(~context_mask[:, None, None], 0.)
        scores = scores.sum(dim=[1, 2, 4])
        ntokens = context_mask.sum(dim=[2]) * n_layers * n_heads
        scores = scores/ntokens
        return scores

    def overwrite_forward_crossattention(self):
        """
        Replace cross-attention forward function, only used to save
        cross-attention scores.
        """
        for mod in self.decoder.block:
            attn = mod.layer[1].EncDecAttention
            attn.forward = types.MethodType(cross_attention_forward, attn)

class EncoderWrapper(torch.nn.Module):
    """
    Encoder Wrapper for T5 Wrapper to obtain a Fusion-in-Decoder model.
    """
    def __init__(self, encoder, use_checkpoint=False):
        super().__init__()

        self.encoder = encoder
        apply_checkpoint_wrapper(self.encoder, use_checkpoint)

    def forward(self, input_ids=None, attention_mask=None, **kwargs,):
        # total_length = n_passages * passage_length
        bsz, total_length = input_ids.shape
        passage_length = total_length // self.n_passages
        input_ids = input_ids.view(bsz*self.n_passages, passage_length)
        attention_mask = attention_mask.view(bsz*self.n_passages, passage_length)
        outputs = self.encoder(input_ids, attention_mask, **kwargs)
        outputs = (outputs[0].view(bsz, self.n_passages*passage_length, -1), ) + outputs[1:]
        return outputs

class CheckpointWrapper(torch.nn.Module):
    """
    Wrapper replacing None outputs by empty tensors, which allows the use of
    checkpointing.
    """
    def __init__(self, module, use_checkpoint=False):
        super().__init__()
        self.module = module
        self.use_checkpoint = use_checkpoint

    def forward(self, hidden_states, attention_mask, position_bias, **kwargs):
        if self.use_checkpoint and self.training:
            kwargs = {k: v for k, v in kwargs.items() if v is not None}
            def custom_forward(*inputs):
                output = self.module(*inputs, **kwargs)
                empty = torch.tensor(
                    [],
                    dtype=torch.float,
                    device=output[0].device,
                    requires_grad=True)
                output = tuple(x if x is not None else empty for x in output)
                return output

            output = torch.utils.checkpoint.checkpoint(
                custom_forward,
                hidden_states,
                attention_mask,
                position_bias
            )
            output = tuple(x if x.size() != 0 else None for x in output)
        else:
            output = self.module(hidden_states, attention_mask, position_bias, **kwargs)
        return output

def apply_checkpoint_wrapper(t5stack, use_checkpoint):
    """
    Wrap each block of the encoder to enable checkpointing.
    """
    block = []
    for mod in t5stack.block:
        wrapped_mod = CheckpointWrapper(mod, use_checkpoint)
        block.append(wrapped_mod)
    block = nn.ModuleList(block)
    t5stack.block = block

In [ ]:
# !pip install evaluate

In [ ]:
from src.data import BinaryCustomDatasetShuffle

In [ ]:
from accelerate import Accelerator
from pprint import pprint
from tqdm import tqdm

In [ ]:
import json
import math
import os
import logging
import sys
import evaluate
from util import utils

import transformers
import torch
import numpy as np
import random
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoModelForSequenceClassification, 
    AutoModel, 
    AutoConfig, 
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
    get_scheduler,
)
from util.arguments import ModelArguments, DataTrainingArguments 
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
class BinaryCustomDatasetShuffle(torch.utils.data.Dataset):
    def __init__(self, instances, tokenizer, max_length, shuffle = False):
        if shuffle:
            random.shuffle(instances)
        self.instances = instances
        self.tokenizer = tokenizer
        self.sep_token = tokenizer.sep_token
        self.max_length = max_length

    def __len__(self):
        return len(self.instances)

    def __getitem__(self, idx):
        input_ = 'question: ' + self.instances[idx]['question'] + \
                 ' title: ' + self.instances[idx]['ctx']['title'] + \
                 ' context : ' + self.instances[idx]['ctx']['text']
        output = self.tokenizer(
            input_,
            # return_tensors="pt", will be applied later through collator
            # padding=True, will be padded later through collate
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_length)

        item = {key: val for key, val in output.items()}
        # item['labels'] = torch.tensor(int(self.instances[idx]['em']))
        item['labels'] = int(self.instances[idx]['em'])

        return item

In [ ]:
report_to = 'wandb'
output_dir = '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/scratch_test'

In [ ]:
# accelerator = Accelerator(log_with=args.report_to, logging_dir=args.output_dir) if args.with_tracking else Accelerator()
#     )

In [ ]:
accelerator = Accelerator(logging_dir=output_dir) 

In [ ]:
accelerator.device

In [ ]:
model_name_or_path = 'roberta-large'
num_labels = 2

In [ ]:
config = AutoConfig.from_pretrained(model_name_or_path, num_labels=num_labels)

In [ ]:
pprint(config)

In [ ]:
print(config.num_labels)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [ ]:
ignore_mismatched_sizes = True

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
        model_name_or_path,
        config=config,
        ignore_mismatched_sizes=ignore_mismatched_sizes,
    )

In [ ]:
train_file = '/data/philhoon-relevance/binary-classification/\
NQ-DEV-DPR/5-fold/1/binary_data/binary_ex_ctx100id_split_train_1_partial.json'
eval_file = '/data/philhoon-relevance/binary-classification/\
NQ-DEV-DPR/5-fold/1/binary_data/binary_ex_ctx100id_split_train_1_partial.json'

In [ ]:
train_data = utils.open_json(train_file)
eval_data = utils.open_json(eval_file)

In [ ]:
max_length = 200
shuffle = False

In [ ]:
train_dataset = BinaryCustomDatasetShuffle(train_data, tokenizer = tokenizer, \
                                           max_length = max_length, shuffle = shuffle)

In [ ]:
eval_dataset = BinaryCustomDatasetShuffle(eval_data, tokenizer = tokenizer, \
                                           max_length = max_length, shuffle = shuffle)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)

In [ ]:
batch_size = 8

In [ ]:
train_dataloader = DataLoader(train_dataset,
                              shuffle = True,
                              collate_fn=data_collator,
                              batch_size=batch_size,
)

In [ ]:
eval_dataloader = DataLoader(eval_dataset,
                              shuffle = True,
                              collate_fn=data_collator,
                              batch_size=batch_size,
)

In [ ]:
no_decay = ["bias", "LayerNorm.weight"]

In [ ]:
weight_decay = 0.0

In [ ]:
optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]

In [ ]:
optimizer_grouped_parameters[0]["weight_decay"]

In [ ]:
learning_rate=5e-5

In [ ]:
optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=learning_rate)

In [ ]:
lr_scheduler_type='linear'
num_warmup_steps = 0
# max_train_steps = 
num_train_epochs = 5
gradient_accumulation_steps = 1

In [ ]:
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)

In [ ]:
max_train_steps = num_train_epochs * num_update_steps_per_epoch

In [ ]:
max_train_steps

In [ ]:
lr_scheduler = get_scheduler(
        name=lr_scheduler_type,
        optimizer=optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=max_train_steps,
)

In [ ]:
accelerator.prepare(
        model, optimizer, train_dataloader, eval_dataloader, lr_scheduler
    )

In [ ]:
metric = evaluate.load("accuracy")

In [ ]:
per_device_train_batch_size = 8

In [ ]:
total_batch_size = per_device_train_batch_size * accelerator.num_processes * gradient_accumulation_steps

In [ ]:
total_batch_size

In [ ]:
progress_bar = tqdm(range(max_train_steps), disable=not accelerator.is_local_main_process)

In [ ]:
num_train_epochs

In [ ]:
starting_epoch = 0
with_tracking = True

In [ ]:
checkpointing_steps = 50

In [ ]:
for epoch in range(starting_epoch, num_train_epochs):
    model.train()
    if with_tracking:
        total_loss = 0
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        
        if with_tracking:
            total_loss += loss.detach().float()
            
        loss = loss / gradient_accumulation_steps
        accelerator.backward(loss)
        
        if step % args.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
            completed_steps += 1
            
        if isinstance(checkpointing_steps, int):
            if completed_steps % checkpointing_steps == 0:
                output_dir = f"step_{completed_steps }"
                if output_dir is not None:
                    output_dir = os.path.join(args.output_dir, output_dir)
                accelerator.save_state(output_dir)
        if completed_steps >= args.max_train_steps:
                break
                
                
    model.eval()
    samples_seen = 0
    for step, batch in enumerate(eval_dataloader):
         with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=-1) 
        predictions, references = accelerator.gather((predictions, batch["labels"]))
        
        if accelerator.num_processes > 1:
            if step == len(eval_dataloader) - 1:
                predictions = predictions[: len(eval_dataloader.dataset) - samples_seen]
                references = references[: len(eval_dataloader.dataset) - samples_seen]
            else:
                samples_seen += references.shape[0]
        
        metric.add_batch(
                predictions=predictions,
                references=references,
            )
        
        eval_metric = metric.compute()
        logger.info(f"epoch {epoch}: {eval_metric}")
        
        if args.with_tracking:
            accelerator.log(
                {
                    "accuracy" : eval_metric,
                    "train_loss": total_loss.item() / len(train_dataloader),
                    "epoch": epoch,
                    "step": completed_steps,
                },
                step=completed_steps,
            )


In [ ]:
metric_acc = evaluate.load("accuracy")
metric_pre = evaluate.load('precision')
metric_re = evaluate.load('recall')
metric_f1 = evaluate.load('f1')

In [ ]:
metric_acc

In [ ]:
accelerator.num_processes

In [ ]:
parser = HfArgumentParser(
    (ModelArguments, DataTrainingArguments, TrainingArguments)
)
args = ["--model_name_or_path", 'allenai/longformer-large-4096', '--output_dir', './']
model_args, data_args, training_args = parser.parse_args_into_dataclasses(args)


In [ ]:
logger = logging.getLogger(__name__)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

# Log on each process the small summary:
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")

In [ ]:
# Detecting last checkpoint.
last_checkpoint = None
if os.path.isdir(training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. "
            "Use --overwrite_output_dir to overcome."
        )
    elif last_checkpoint is not None:
        logger.info(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )


In [ ]:
# Set seed before initializing model.
set_seed(training_args.seed)

In [ ]:
config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        num_labels=model_args.num_labels,
    )
tokenizer = AutoTokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    )
model = AutoModelForSequenceClassification.from_pretrained(
    model_args.model_name_or_path,
    config=config,
)

In [ ]:
if training_args.do_train:
    instances, cut_off, total_questions = preprocessing_data(
        data_args.train_file, 
        data_args.sample_size, 
        data_args.position)
    
    train_instance = instances[data_args.dev_size:]
    dev_instance = instances[:data_args.dev_size]
    
    train_dataset = CustomDataset(train_instance, 
                               tokenizer, 
                               model_args.max_seq_length)
    dev_dataset = CustomDataset(train_instance, 
                               tokenizer, 
                               model_args.max_seq_length)
    
    # Log a few random samples from the training set:
    for index in random.sample(range(len(train_dataset)), 3):
        logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

if training_args.do_eval:
    instances, cut_off, total_questions = preprocessing_data(
        data_args.test_file, 
        data_args.sample_size, 
        data_args.position)
    
    test_dataset = CustomDataset(instances, 
                               tokenizer, 
                               model_args.max_seq_length)
    
    


In [ ]:
# Get the metric function
metric = evaluate.load("xnli")

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)


In [ ]:
 # Initialize Trainer
data_collator = DataCollatorWithPadding(
    tokenizer, 
    pad_to_multiple_of=8,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=eval_dataset if training_args.do_train else None,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=30)]
)

# Training
if training_args.do_train:
    checkpoint = None
    if training_args.resume_from_checkpoint is not None:
        checkpoint = training_args.resume_from_checkpoint
    elif last_checkpoint is not None:
        checkpoint = last_checkpoint
    train_result = trainer.train(resume_from_checkpoint=checkpoint)
    metrics = train_result.metrics
    max_train_samples = (
        data_args.max_train_samples if data_args.max_train_samples is not None else len(train_dataset)
    )
    metrics["train_samples"] = min(max_train_samples, len(train_dataset))

    trainer.save_model()  # Saves the tokenizer too for easy upload

    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()
    
# Evaluation
if training_args.do_eval:
    logger.info("*** Evaluate ***")
    metrics = trainer.evaluate(eval_dataset=eval_dataset)

    max_eval_samples = data_args.max_eval_samples if data_args.max_eval_samples is not None else len(eval_dataset)
    metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))

    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)

In [ ]:
metric

In [ ]:
data_args.dataset_name = a
    

In [ ]:
data.max_seq_length

In [ ]:
training_args.fp16

In [ ]:
bb

In [ ]:
def main():
    parser = HfArgumentParser(
        (ModelArguments, DataTrainingArguments, TrainingArguments)
    )
    
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()
    
    